# Vektordatenbank Suche
Ein Notebook nur spezialisiert auf die Suche in der Vektordatenbank

In [1]:
from classes import *

workers = get_test_data()

Einige spezialisierte Test-Mitarbeiter hinzufügen

In [2]:
test_mitarbeiter = [
    MitarbeiterSkills(
        name="Boris Backend",
        technologien=[
            TechnologieWissen(name="Java", letzte_verwendung=0),
            TechnologieWissen(name="Docker", letzte_verwendung=0),
            TechnologieWissen(name="SQL", letzte_verwendung=0),
            TechnologieWissen(name="MongoDB", letzte_verwendung=0),
            TechnologieWissen(name="PostgreSQL", letzte_verwendung=0),
            TechnologieWissen(name="MySQL", letzte_verwendung=0)
        ]
    ),
    MitarbeiterSkills(
        name="Fritz Frontend",
        technologien=[
            TechnologieWissen(name="JavaScript", letzte_verwendung=0),
            TechnologieWissen(name="TypeScript", letzte_verwendung=0),
            TechnologieWissen(name="React", letzte_verwendung=0),
            TechnologieWissen(name="Vue.js", letzte_verwendung=0),
            TechnologieWissen(name="Angular", letzte_verwendung=0)
        ]
    ),
    MitarbeiterSkills(
        name="Tim Technik",
        technologien=[
            TechnologieWissen(name="Azure", letzte_verwendung=0),
            TechnologieWissen(name="AWS", letzte_verwendung=0),
            TechnologieWissen(name="GCP", letzte_verwendung=0),
            TechnologieWissen(name="Kubernetes", letzte_verwendung=0),
            TechnologieWissen(name="Docker", letzte_verwendung=0)
        ]
    )
]

# Füge die Test-Mitarbeiter zu den bestehenden Mitarbeitern hinzu
workers.extend(test_mitarbeiter)

In [3]:
from openai import OpenAI

client = OpenAI()

def embedd(data):
    return client.embeddings.create(input=data, model="text-embedding-3-small").data[0].embedding

In [4]:
import os
import numpy as np

EMBEDDINGS_PATH = "embeddings.npy"

embedded_data = []

if os.path.exists(EMBEDDINGS_PATH):
    print("Lade gespeicherte Embeddings...")
    embedded_data = np.load(EMBEDDINGS_PATH)
else:
    print("Berechne Embeddings und speichere sie...")
    embedded_data = []
    for worker in workers:
        embedded_data.append(embedd(worker.to_embedding()))
    embedded_data = np.array(embedded_data)
    np.save(EMBEDDINGS_PATH, embedded_data)

Lade gespeicherte Embeddings...


In [5]:
import faiss
import numpy as np

dim = len(embedded_data[0])
index = faiss.IndexFlatL2(dim)

embeddings_np = np.array(embedded_data).astype("float32")
index.add(embeddings_np)

## Notitz
Es gibt 2 Ansätze, die man noch probieren könnte.

1. Alle Daten vor dem Embedding in Fließtext konvertieren
2. Query-Vektor in Stichpunkte bzw. konkrete Technologie suche Konvertieren (siehe Excalidraw) 

In [6]:
from embeddings_utils import get_query_embedding

query = "Erfahrung mit Backend Technologien"
query_vector = get_query_embedding(query, embedd)

search_vector = np.array([query_vector]).astype('float32')

erg_anzahl = 5

D, indices = index.search(search_vector, erg_anzahl)

print(f"Distances: {D}")
print(f"Indices: {indices}")

for i in indices[0]:
    print(workers[i])

Distances: [[1.1821351 1.2131362 1.2248633 1.2301035 1.2459797]]
Indices: [[199 106 228  26 152]]
Bianca Grebe (Express.js, PHP, Flask, Unreal Engine, Azure)
Manuel Schuster (Kubernetes, Java, Docker, Terraform, JavaScript)
Boris Backend (Java, Docker, SQL, MongoDB, PostgreSQL, MySQL)
Tom Weiß (Keras, PHP, Google Cloud, Salesforce, Swift)
Kevin Ostermann (AWS, AWS)


In [7]:
import ipywidgets as widgets
from IPython.display import display, clear_output
from embeddings_utils import get_query_embedding
from visualisation import plot_with_query_umap

query_input = widgets.Text(
    value='Erfahrung mit Backend Technologien',
    placeholder='Suchanfrage eingeben...',
    description='Query:',
    disabled=False
)

output = widgets.Output()

def on_query_change(change):
    with output:
        clear_output(wait=True)
        query = change['new']
        query_vector = get_query_embedding(query, embedd)
        search_vector = np.array([query_vector]).astype('float32')
        erg_anzahl = 5
        D, indices = index.search(search_vector, erg_anzahl)
        # Use string representation of each worker for hover info
        hover_texts = [str(w) for w in workers]
        plot_with_query_umap(embeddings_np, query_vector, highlight_indices=indices[0], hover_texts=hover_texts)
        print(f"Distances: {D}")
        print(f"Indices: {indices}")
        for i in indices[0]:
            print(workers[i])

query_input.observe(on_query_change, names='value')
display(query_input, output)
# Initial plot
on_query_change({'new': query_input.value})

Text(value='Erfahrung mit Backend Technologien', description='Query:', placeholder='Suchanfrage eingeben...')

Output()